In [ ]:
from tools.load_nc import load_nc_sat, load_nc_sinmod, load_nc_insitu
import matplotlib.pyplot as plt
from tools import grid_conversion
import numpy as np
from tools.compare_tools import *
from scipy.spatial import cKDTree
import cnn
import importlib
importlib.reload(cnn)

In [ ]:
insitu_fpath = 'D:/master/data/compare/satellite2/phys_200.nc'
sinmod_fpath = 'D:/master/data/compare/sinmod/samples_2017.06.01_nonoverlap.nc'
# Load insitu netcdf data
lons_insitu,lats_insitu,sst_insitu,ssl_insitu,sal_insitu,uvel_insitu,vvel_insitu,t_insitu =  load_nc_insitu(insitu_fpath)
# Get SINMOD data
x_sinmod,y_sinmod,_,lons_sinmod,lats_sinmod,sst_sinmod,ssl_sinmod,sal_sinmod,uvel_sinmod,vvel_sinmod =  load_nc_sinmod(sinmod_fpath)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

sizes = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
accuracies = [0.942985075712204, 0.9488059729337692, 0.9548507511615754, 0.9552238792181015, 0.9605970174074173, 0.9549253761768342, 0.9560447782278061, 0.9571641772985459, 0.9517910420894623, 0.9565671622753144, 0.9526865690946579, 0.9502985060214997, 0.950074627995491, 0.9478358209133149, 0.9454477608203888, 0.9406716406345368]
sigma = [0.009351913739716677, 0.014269941662587389, 0.007370702153183762, 0.0072660657355606, 0.0077611942704084, 0.010861730695364735, 0.00571224730545424, 0.009226001121706883, 0.00890659141791928, 0.007416273702419764, 0.010732784150351846, 0.0168339337271264, 0.009492289952609938, 0.009194259288595437, 0.009585710047888923, 0.02138485614179754]
time = [8.703298997879028, 9.369982755184173, 10.06664674282074, 10.54482423365116, 10.90820716381073, 11.546471973260244, 12.144830793993814, 12.573934230208398, 16.696370553970336, 17.530485967795055, 18.14318478703499, 24.768352270126343, 28.91509838104248, 30.110700178146363, 31.44364127715429, 31.676390677690506]


accuracies = np.multiply(accuracies, 100)
sigma = np.multiply(sigma, 100)

fig = plt.figure(figsize=(10,6))
#ax = fig.add_subplot(111)

# use a gray background
ax = plt.axes(facecolor='#E6E6E6')
ax.set_axisbelow(True)
# hide axis spines
for spine in ax.spines.values():
    spine.set_visible(False)

plt.grid(color='w', linestyle='solid')

#sizes = np.arange(5, 101, 5)
ax.errorbar(sizes, accuracies, sigma, linestyle='--', marker='.')
for i, v in enumerate(accuracies):
    ax.text(sizes[i]+0.3, v-0.2, "%0.1fs" %time[i])#, ha="center")
plt.xlim([0,85])
plt.xlabel('Input size', size=12)
plt.ylabel('accuracy [%]', size=12)




In [ ]:

#x_bnds = [1000000, 2000000]
#y_bnds = [1000000, 2100000]
x_bnds = [1300000, 2000000]
y_bnds = [1000000, 1600000]

# Get a polar steregraphic projection with cartesian irregular curvilinear grid for satellite data 
x_insitu_2d, y_insitu_2d = grid_conversion.bl2xy(*np.meshgrid(lons_insitu, lats_insitu), FE=3254000, FN=2560000, xy_res=1, slon=58, SP=60)

# Interpolate data and the curvilienar grid axes such that it has a higher resolution than SINMOD
sst_insitu = interp2d_masked(sst_insitu, lons_insitu, lats_insitu, 8, 'linear')
ssl_insitu = interp2d_masked(ssl_insitu, lons_insitu, lats_insitu, 8, 'linear')
sal_insitu = interp2d_masked(sal_insitu, lons_insitu, lats_insitu, 8, 'linear')
uvel_insitu = interp2d_masked(uvel_insitu, lons_insitu, lats_insitu, 8, 'linear')
vvel_insitu = interp2d_masked(vvel_insitu, lons_insitu, lats_insitu, 8, 'linear')
x_insitu = interp2d_masked(x_insitu_2d.T, lons_insitu, lats_insitu, 8, 'linear')
y_insitu = interp2d_masked(y_insitu_2d.T, lons_insitu, lats_insitu, 8, 'linear')

x_idxs = np.where((x_sinmod >= x_bnds[0]) & (x_sinmod <= x_bnds[1]))[0]
y_idxs = np.where((y_sinmod >= y_bnds[0]) & (y_sinmod <= y_bnds[1]))[0]
x_sinmod = np.ma.array(x_sinmod[x_idxs])
y_sinmod = np.ma.array(y_sinmod[y_idxs])
lons_sinmod = np.ma.array(lons_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])
lats_sinmod = np.ma.array(lats_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])
sst_sinmod = np.ma.array(sst_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])
ssl_sinmod = np.ma.array(ssl_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])
sal_sinmod = np.ma.array(sal_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])
uvel_sinmod = np.ma.array(uvel_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])
vvel_sinmod = np.ma.array(vvel_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])

In [ ]:
# Create cKDTree object to represent the full flattened cartesian coordinates of the satellite data grid
tree = cKDTree(list(zip(x_insitu.flatten(), y_insitu.flatten())))

In [ ]:

# Find indices of the satellite data nearest to the SINMOD coordinates we use
x_sinmod_2d, y_sinmod_2d = np.meshgrid(x_sinmod, y_sinmod)
d, inds = tree.query(list(zip(x_sinmod_2d.ravel(), y_sinmod_2d.ravel())), k = 1)

# Extract satellite data by query indexes to get same grid axes and apply the same landmass mask
sinmod_mask = np.ma.getmask(ssl_sinmod)
sst_insitu = np.ma.masked_where(sinmod_mask, sst_insitu.flatten()[inds].reshape(uvel_sinmod.T.shape).T)
ssl_insitu = np.ma.masked_where(sinmod_mask, ssl_insitu.flatten()[inds].reshape(uvel_sinmod.T.shape).T)
sal_insitu = np.ma.masked_where(sinmod_mask, sal_insitu.flatten()[inds].reshape(uvel_sinmod.T.shape).T)
uvel_insitu = np.ma.masked_where(sinmod_mask, uvel_insitu.flatten()[inds].reshape(uvel_sinmod.T.shape).T)
vvel_insitu = np.ma.masked_where(sinmod_mask, vvel_insitu.flatten()[inds].reshape(uvel_sinmod.T.shape).T)

# Rotate velocities to fit the shifted pole stereographic projection
uvel_insitu, vvel_insitu = rotate_vector(uvel_insitu, vvel_insitu, lons_sinmod, 58)

insitu_data = [lons_sinmod, lats_sinmod, x_sinmod, y_sinmod, ssl_insitu, uvel_insitu, vvel_insitu]

In [ ]:
#fig, ax = plt.subplots(1,1,figsize=(12,6), dpi=150)

#ax.pcolormesh(x_sinmod, y_sinmod, ssl_insitu.T, cmap='rainbow')
#ax.streamplot(x_sinmod, y_sinmod, uvel_insitu.T, vvel_insitu.T, density=8) 
#ax.set_xlim([x_sinmod[0], x_sinmod[-1]])
#ax.set_ylim([y_sinmod[0], y_sinmod[-1]])

In [ ]:
#win_sizes = [((int(7), int(5)), 2, 2),((int(9), int(7)), 2, 2),((int(12), int(9)), 3, 3), ((int(16), int(12)), 4, 4)]
win_sizes = [((int(8), int(6)), 2, 2),((int(11), int(8)), 3, 2),((int(14), int(11)), 3, 3)]
#model_fpath = 'D:/master/models/2016/cnn_mult_velocities_9652.h5'
model_fpath = 'D:/master/models/2016/cnn_mult_velocities_9652.h5'
cyc, acyc = cnn.cnn_predict_grid(insitu_data, win_sizes, 0.92, model_fpath, storedir='insitu1')

In [ ]:
# Group all predicted boxes according to overlap and number of rectangles in a group
cyc, _ = cv2.groupRectangles(rectList=cyc, groupThreshold=2, eps=0.1)
acyc, _ = cv2.groupRectangles(rectList=acyc, groupThreshold=2, eps=0.1)

In [ ]:
x,y,lons,lats,uvel,vvel,ssl = x_sinmod,y_sinmod,lons_sinmod,lats_sinmod,uvel_insitu,vvel_insitu,ssl_insitu

OW_start = -0.05
OW,vorticity,OW_mask,OW_cyc_mask,OW_acyc_mask = calc_OW(lons,lats,uvel,vvel,OW_start)

# We label all unique mask clusters
OW_mask_labeled = measure.label(OW_mask)
cyc_mask_labeled = measure.label(OW_cyc_mask)
acyc_mask_labeled = measure.label(OW_acyc_mask)

# Make the predicted boxes encompass the full cyclone or anti-cyclone clusters and return info about the eddy
cyc_ctrIdxs, cyc_minOW, cyc_new= investigate_cluster(cyc, OW, cyc_mask_labeled, 'cyclone')
acyc_ctrIdxs, acyc_minOW, acyc_new = investigate_cluster(acyc, OW, acyc_mask_labeled, 'anti-cyclone')

# Eddy census dataframe to hold information about each eddy
cyc_census_df = set_eddy_census(cyc_mask_labeled, cyc_ctrIdxs, cyc_new, cyc_minOW, lons, lats, x, y, uvel, vvel, meastype='SINMOD')
acyc_census_df = set_eddy_census(acyc_mask_labeled, acyc_ctrIdxs, acyc_new, acyc_minOW, lons, lats, x, y, uvel, vvel, meastype='SINMOD')

cyc_plotcoords = box2coords(x, y, cyc_new)
acyc_plotcoords = box2coords(x, y, acyc_new)



In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12,6), dpi=150)

ax.pcolormesh(x, y, ssl.T, cmap='rainbow')
ax.streamplot(x, y, uvel.T, vvel.T, density=8) 
#ax.quiver(x,y,uvel.T,vvel.T,scale=4)
#ax.set_xlim([1750000, 2000000])
#ax.set_ylim([1200000, 1500000])
plot_eddies(ax, cyc_plotcoords, 'r', numbered=False) 
plot_eddies(ax, acyc_plotcoords, 'b', numbered=False) 

In [ ]:
dirpath = 'C:/Users/47415/Master/TTK-4900-Master/data/training_data/2016/'

ssl = []
uvel = []
vvel = []

data = np.empty((4, 4), dtype=object)

# anti, nothing, nothing, cyclone 1509
for i, eddyId in enumerate((697, 1519, 651, 702)):
    for j, fpath in enumerate((dirpath+'ssl_train.npz', dirpath+'sst_train.npz', dirpath+'uvel_train.npz', dirpath+'vvel_train.npz')):
        with np.load(fpath, allow_pickle=True) as h5f:
            data[i,j] = h5f['arr_0'][eddyId,0]

fig, ax = plt.subplots(4,4,figsize=(12,10))

ax[0,0].pcolormesh(data[0,0].T, cmap='rainbow') 
ax[0,0].set_title("SSH", fontsize=16)
ax[0,0].set_ylabel('Cyclone', fontsize=16)
ax[0,0].axes.xaxis.set_ticks([])
ax[0,0].axes.yaxis.set_ticks([])

ax[0,1].pcolormesh(data[0,1].T, cmap='rainbow')
ax[0,1].axes.xaxis.set_ticks([])
ax[0,1].axes.yaxis.set_ticks([])
ax[0,1].set_title("SST", fontsize=16)

ax[0,2].pcolormesh(data[0,2].T, cmap='rainbow')
ax[0,2].axes.xaxis.set_ticks([])
ax[0,2].axes.yaxis.set_ticks([])
ax[0,2].set_title("uvel", fontsize=16)

ax[0,3].pcolormesh(data[0,3].T, cmap='rainbow')
ax[0,3].axes.xaxis.set_ticks([])
ax[0,3].axes.yaxis.set_ticks([])
ax[0,3].set_title("vvel", fontsize=16)

ax[1,0].pcolormesh(data[1,0].T, cmap='rainbow') 
ax[1,0].set_ylabel('Cyclone', fontsize=16)
ax[1,0].axes.xaxis.set_ticks([])
ax[1,0].axes.yaxis.set_ticks([])

ax[1,1].pcolormesh(data[1,1].T, cmap='rainbow')
ax[1,1].axes.xaxis.set_ticks([])
ax[1,1].axes.yaxis.set_ticks([])

ax[1,2].pcolormesh(data[1,2].T, cmap='rainbow')
ax[1,2].axes.xaxis.set_ticks([])
ax[1,2].axes.yaxis.set_ticks([])

ax[1,3].pcolormesh(data[1,3].T, cmap='rainbow')
ax[1,3].axes.xaxis.set_ticks([])
ax[1,3].axes.yaxis.set_ticks([])

ax[2,0].pcolormesh(data[2,0].T, cmap='rainbow') 
ax[2,0].set_ylabel('Anti-cyclone', fontsize=16)
ax[2,0].axes.xaxis.set_ticks([])
ax[2,0].axes.yaxis.set_ticks([])

ax[2,1].pcolormesh(data[2,1].T, cmap='rainbow')
ax[2,1].axes.xaxis.set_ticks([])
ax[2,1].axes.yaxis.set_ticks([])

ax[2,2].pcolormesh(data[2,2].T, cmap='rainbow')
ax[2,2].axes.xaxis.set_ticks([])
ax[2,2].axes.yaxis.set_ticks([])

ax[2,3].pcolormesh(data[2,3].T, cmap='rainbow')
ax[2,3].axes.xaxis.set_ticks([])
ax[2,3].axes.yaxis.set_ticks([])

ax[3,0].pcolormesh(data[3,0].T, cmap='rainbow') 
ax[3,0].set_ylabel('Anti-cyclone', fontsize=16)
ax[3,0].axes.xaxis.set_ticks([])
ax[3,0].axes.yaxis.set_ticks([])

ax[3,1].pcolormesh(data[3,1].T, cmap='rainbow')
ax[3,1].axes.xaxis.set_ticks([])
ax[3,1].axes.yaxis.set_ticks([])

ax[3,2].pcolormesh(data[3,2].T, cmap='rainbow')
ax[3,2].axes.xaxis.set_ticks([])
ax[3,2].axes.yaxis.set_ticks([])

ax[3,3].pcolormesh(data[3,3].T, cmap='rainbow')
ax[3,3].axes.xaxis.set_ticks([])
ax[3,3].axes.yaxis.set_ticks([])

plt.tight_layout()